In [35]:
#!pip install fiona
import fiona
import pandas as pd
import shapely
import geopandas
from shapely.geometry import shape,mapping, Point, Polygon, MultiPolygon
import matplotlib.pyplot as plt


In [3]:
df = pd.read_csv('../Capstone/csv/trip_data_6.csv', error_bad_lines=False)
dff = df.copy()

In [6]:
# https://towardsdatascience.com/geospatial-operations-at-scale-with-dask-and-geopandas-4d92d00eb7e8
def assign_taxi_zones(df, lon_var, lat_var, locid_var):

    import geopandas
    from shapely.geometry import Point

    localdf = df[[lon_var, lat_var]].copy()
    
    localdf[lon_var] = localdf[lon_var].fillna(value=0.)
    localdf[lat_var] = localdf[lat_var].fillna(value=0.)
    
    shape_df = geopandas.read_file('../Capstone/taxi_zones/taxi_zones.shp')
    shape_df = shape_df.to_crs({'init': 'epsg:4326'})
    try:
        local_gdf = geopandas.GeoDataFrame(
            localdf, crs={'init': 'epsg:4326'},
            geometry=[Point(xy) for xy in
                      zip(localdf[lon_var], localdf[lat_var])])

        local_gdf = geopandas.sjoin(
            local_gdf, shape_df, how='left', op='within')

        return local_gdf.OBJECTID.rename(locid_var)
    except ValueError as ve:
        print(ve)
        print(ve.stacktrace())
        series = localdf[lon_var]
        series = np.nan
        return series

In [7]:
dff = dff.join(assign_taxi_zones(dff, " pickup_longitude", " pickup_latitude", "pickup_taxizone_id"))
dff = dff.join(assign_taxi_zones(dff, " dropoff_longitude", " dropoff_latitude", "dropoff_taxizone_id"))

In [9]:
dff.to_csv('result.csv',index=False)

In [10]:
dff.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_taxizone_id,dropoff_taxizone_id
0,2013009441,2013013593,VTS,1,NaN,2013-06-01 00:00:00,2013-06-01 00:05:00,1,300,0.74,-73.972008,40.749767,-73.968109,40.756382,233.0,229.0
1,2013012081,2013019632,VTS,1,NaN,2013-06-01 00:00:00,2013-06-01 00:16:00,1,960,3.52,-73.984909,40.779106,-73.944702,40.803219,143.0,41.0
2,2013006133,2013029678,VTS,1,NaN,2013-06-01 00:00:00,2013-06-01 00:10:00,1,600,2.34,-73.992043,40.725639,-73.990120,40.750992,79.0,100.0
3,2013007864,2013007860,VTS,1,NaN,2013-06-01 00:00:00,2013-06-01 00:08:00,1,480,1.63,-73.982071,40.743259,-73.990967,40.723995,170.0,79.0
4,2013010123,2013010123,VTS,1,NaN,2013-06-01 00:00:00,2013-06-01 00:19:00,4,1140,5.98,-73.999809,40.732849,-73.962357,40.800049,113.0,24.0
